# Métodos Montecarlo Fing 2022 - Entrega 1

**Autor:** Carlos M. Martinez, marzo-abril 2022.

**Email:** carlosm@fing.edu.uy, carlos@cagnazzo.uy



Supongamos que para construir una casa debemos efectuar la siguiente lista de tareas:

- T1 - cimientos - tiempo aleatorio uniforme entre 40 y 56 hs. 
- T2 - contrapiso - tiempo aleatorio uniforme entre 24 y 32 hs. 
- T3 - paredes - tiempo aleatorio uniforme entre 20 y 40 hs.
- T4 - techo - tiempo aleatorio uniforme entre 16 y 48 hs.
- T5 - instalación sanitaria - tiempo aleatorio uniforme entre 10 y 30 hs.
- T6 - instalación el eléctrica - tiempo aleatorio uniforme entre 15 y 30 hs. 
- T7 - cerramientos - tiempo aleatorio uniforme entre 20 y 25 hs.
- T8 - pintura - tiempo aleatorio uniforme entre 30 y 50 hs.
- T9 - revestimientos sanitarios - tiempo aleatorio uniforme entre 40 y 60 hs.
- T10 - limpieza final - tiempo aleatorio uniforme entre 8 y 16 hs.

Hay ciertas dependencias que implican que una tarea no puede comenzar hasta haberse terminado otra previa:

- T2, T3 dependen de 1.
- T4 depende de 2 y 3
- T5 depende de 2 y 3
- T6 depende de 3
- T7 depende de 3
- T8 dependede 4,5,6 y7
- T9 depende de 5
- T10 depende de 7, 8 y 9

## Entrega 1 

**Ejercicio 2.1**

Implementar un programa que reciba como par ́ametros de l ́ınea de comando (o pregunte en pantalla) la cantidad de replicaciones n a realizar, y emplee Monte Carlo para calcular (e imprimir) la estimaci ́on del tiempo total promedio desde que se comienza la obra hasta que se finaliza la misma, y la desviaci ́on est ́andar de este estimador.

In [17]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import random

# Estructura de datos para las tareas:

TareasDB = ['']*10

TareasDB[1] = {
    'dur': (40,56), # intervalo de duración en horas
    'dep': []       # tareas de la que depende para poder comenzar
}

TareasDB[2] = {
    'dur': (24,32), # intervalo de duración en horas
    'dep': [1]       # tareas de la que depende para poder comenzar
}

TareasDB[3] = {
    'dur': (20,40), # intervalo de duración en horas
    'dep': [1]       # tareas de la que depende para poder comenzar
}

TareasDB[4] = {
    'dur': (16,48), # intervalo de duración en horas
    'dep': [2,3]       # tareas de la que depende para poder comenzar
}


In [23]:
# Generar muestras de las duraciones de las tareas
TSim = [0] * 10

random.seed()
for i in range(1, 5):
    TSim[i] = random.randint(TareasDB[i]['dur'][0], TareasDB[i]['dur'][1])
    
TSim

[0, 55, 24, 28, 42, 0, 0, 0, 0, 0]

In [30]:
# Calculo del tiempo de construcción

# Alg1 , "ir para atras"
# Tomo la tarea final
# Le sumo la duración max(tareas pre condicion)
# repito para la tarea que tenía ese max

def calculoTiempoDeConstruccion(wTSim, wTDB, n):
    Ttotal = 0
    
    if n == 1:
        return wTSim[1]
    else:
        max_precond = 0
        t_max_precond = 0
        for j in wTDB[n]['dep']:
            if wTSim[j] > max_precond:
                max_precond = wTSim[j]
                t_max_precond = j
                print(max_precond, t_max_precond)
            # end if
        # end for
            
        Ttotal = wTSim[n] + max_precond + calculoTiempoDeConstruccion(wTSim, wTDB, t_max_precond) 
    
    return Ttotal

In [31]:
calculoTiempoDeConstruccion(TSim, TareasDB, 4)

24 2
28 3
55 1


55

## Conclusiones

In [15]:
# TBW*